<a href="https://colab.research.google.com/github/HimanshuMK/Captcha-Recognition-Model/blob/main/Captcha_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Importing Libraries

In [2]:
import numpy as np
import os
import cv2 #OpenCV(Open Source computer vision lib), containg CV algos
import string
import matplotlib.pyplot as plt #for graphs

### Importing data

In [3]:
data_dir = '/content/drive/MyDrive/OCR_model/samples'
labels = []
images = []

### Extracting Images and labels From data

In [4]:
for filename in os.listdir(data_dir):
    # read image
    img = cv2.imread(os.path.join(data_dir, filename), cv2.IMREAD_GRAYSCALE)
    images.append(img)

    # extract labels from filename
    label = filename.split('.')[0]
    labels.append(label)

In [5]:
print(np.array(images).shape)
print(np.array(labels).shape)

(1070, 50, 200)
(1070,)


#### For Encoding Purpose

In [6]:
characters= string.ascii_lowercase + string.digits # All symbols captcha can contain
nchar = len(characters) #total number of char possible

### Encoding Functions

In [7]:
char_to_num = {}
for idx, char in enumerate(characters):
    char_to_num[char] = idx

# number to character conversion
num_to_char = {}
for char, idx in char_to_num.items():
    num_to_char[idx] = char

Since there are 1070 images. <br>
We will define first shapes of X and y <br>



In [8]:
n = len(images)
print(n)

1070


In [9]:
#defining size of image
imgshape=(50,200,1) #50-height, 200-width, 1-no of channels

### Creating X and y for Model

In [10]:
X = np.zeros((n, 50, 200, 1))  # n * 50 * 200 array with all entries 0
y = np.zeros((5, n, nchar))    # 5 * n * 36 (5 letters in captcha) with all entries 0

In [11]:
for i, (img, label) in enumerate(zip(images, labels)):
    if len(label) < 6:  # captcha is not more than 5 letters
        img = img / 255.0  # scales the image between 0 and 1
        img = np.reshape(img, (50, 200, 1))  # reshapes image to width 200, height 50, channel 1

        target = np.zeros((5, nchar))  # creates an array of size 5*36 with all entries 0

        for j, k in enumerate(label):
            # j iterates from 0 to 4 (5 letters in captcha)
            # k denotes the letter in captcha which is to be scanned
            index = char_to_num[k]  # index stores the position of letter k of captcha in the char_to_num dictionary
            target[j, index] = 1  # replaces 0 with 1 in the target array at the position of the letter in captcha

        X[i] = img  # stores all the images
        y[:, i] = target  # stores all the info about the letters in captcha of all images

In [12]:
print(X.shape)
print(y.shape)

(1070, 50, 200, 1)
(5, 1070, 36)


Since now our X and y are ready, <br>
we will Divide it in training and testing set

Train Test Split

In [13]:
import numpy as np

# Set a random seed for reproducibility
np.random.seed(42)

# Shuffle the indices
indices = np.arange(X.shape[0])
np.random.shuffle(indices)

# Shuffle the data accordingly
X_shuffled = X[indices]
y_shuffled = y[:, indices]

# Split the data into training and test sets
train_size = 970
X_train, y_train = X_shuffled[:train_size], y_shuffled[:, :train_size]
X_test, y_test = X_shuffled[train_size:], y_shuffled[:, train_size:]

# Verify the shapes
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

# (970, 50, 200, 1)
# (5, 970, 36)
# (100, 50, 200, 1)
# (5, 100, 36)


(970, 50, 200, 1)
(5, 970, 36)
(100, 50, 200, 1)
(5, 100, 36)


# Creating a Model

In [14]:
from keras import layers #for building layers of neural net
from keras.models import Model
from keras.models import load_model
from keras import callbacks #for training logs, saving to disk periodically

In [15]:
#create model
def createmodel():
    img = layers.Input(shape=imgshape) # Get image as an input of size 50,200,1
    conv1 = layers.Conv2D(16, (3, 3), padding='same', activation='relu')(img) #50*200
    mp1 = layers.MaxPooling2D(padding='same')(conv1)  # 25*100
    conv2 = layers.Conv2D(32, (3, 3), padding='same', activation='relu')(mp1)
    mp2 = layers.MaxPooling2D(padding='same')(conv2)  # 13*50
    conv3 = layers.Conv2D(32, (3, 3), padding='same', activation='relu')(mp2)
    bn = layers.BatchNormalization()(conv3) #to improve the stability of model
    mp3 = layers.MaxPooling2D(padding='same')(bn)  # 7*25

    flat = layers.Flatten()(mp3) #convert the layer into 1-D

    outs = []
    for _ in range(5): #for 5 letters of captcha
        dens1 = layers.Dense(64, activation='relu')(flat)
        drop = layers.Dropout(0.5)(dens1) #drops 0.5 fraction of nodes
        res = layers.Dense(nchar, activation='sigmoid')(drop)

        outs.append(res) #result of layers

    # Compile model and return it
    model = Model(img, outs) #create model
    model.compile(loss='categorical_crossentropy', optimizer='adam',metrics=["accuracy"])
    return model

In [16]:
#Create model
model=createmodel();
model.summary();

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 50, 200, 1)]         0         []                            
                                                                                                  
 conv2d (Conv2D)             (None, 50, 200, 16)          160       ['input_1[0][0]']             
                                                                                                  
 max_pooling2d (MaxPooling2  (None, 25, 100, 16)          0         ['conv2d[0][0]']              
 D)                                                                                               
                                                                                                  
 conv2d_1 (Conv2D)           (None, 25, 100, 32)          4640      ['max_pooling2d[0][0]']   

Trying to fit in that model

In [17]:
#Applying the model
hist = model.fit(X_train, [y_train[0], y_train[1], y_train[2], y_train[3], y_train[4]], batch_size=32, epochs=60, validation_split=0.2)
#batch size- 32 defines no. of samples per gradient update
#Validation split=0.2 splits the training set in 80-20% for training nd testing

Epoch 1/60
25/25 [==============================] - 14s 300ms/step - loss: 18.0751 - dense_1_loss: 3.6128 - dense_3_loss: 3.5889 - dense_5_loss: 3.6385 - dense_7_loss: 3.6048 - dense_9_loss: 3.6301 - dense_1_accuracy: 0.0528 - dense_3_accuracy: 0.0619 - dense_5_accuracy: 0.0515 - dense_7_accuracy: 0.0567 - dense_9_accuracy: 0.0554 - val_loss: 17.7621 - val_dense_1_loss: 3.5460 - val_dense_3_loss: 3.5523 - val_dense_5_loss: 3.5540 - val_dense_7_loss: 3.5475 - val_dense_9_loss: 3.5622 - val_dense_1_accuracy: 0.0876 - val_dense_3_accuracy: 0.0464 - val_dense_5_accuracy: 0.0722 - val_dense_7_accuracy: 0.0412 - val_dense_9_accuracy: 0.0515
Epoch 2/60
25/25 [==============================] - 10s 380ms/step - loss: 16.8518 - dense_1_loss: 3.3242 - dense_3_loss: 3.4124 - dense_5_loss: 3.3669 - dense_7_loss: 3.3096 - dense_9_loss: 3.4387 - dense_1_accuracy: 0.0735 - dense_3_accuracy: 0.0606 - dense_5_accuracy: 0.0709 - dense_7_accuracy: 0.0644 - dense_9_accuracy: 0.0709 - val_loss: 17.6867 - va

### Saving the Model for furthur use

In [20]:
# Define the save path
save_path = '/content/drive/MyDrive/OCR_model/captcha_model_v2.keras'
# Save the model
model.save(save_path)


## Evaluating Model on Training data

In [18]:
#Loss on training set
#Finding Loss on training set
preds = model.evaluate(X_train, [y_train[0], y_train[1], y_train[2], y_train[3], y_train[4]])
print ("Loss on training set= " + str(preds[0]))

31/31 [==============================] - 2s 65ms/step - loss: 0.4404 - dense_1_loss: 0.0096 - dense_3_loss: 0.0703 - dense_5_loss: 0.0926 - dense_7_loss: 0.1528 - dense_9_loss: 0.1152 - dense_1_accuracy: 0.9979 - dense_3_accuracy: 0.9887 - dense_5_accuracy: 0.9814 - dense_7_accuracy: 0.9742 - dense_9_accuracy: 0.9722
Loss on training set= 0.44039222598075867


## Evaluating Model on Testing data

In [19]:
#Finding loss on test set
preds = model.evaluate(X_test, [y_test[0], y_test[1], y_test[2], y_test[3], y_test[4]])
print ("Loss on testing set= " + str(preds[0]))

4/4 [==============================] - 0s 49ms/step - loss: 1.7590 - dense_1_loss: 0.0089 - dense_3_loss: 0.2264 - dense_5_loss: 0.3160 - dense_7_loss: 0.7913 - dense_9_loss: 0.4163 - dense_1_accuracy: 0.9900 - dense_3_accuracy: 0.9600 - dense_5_accuracy: 0.9200 - dense_7_accuracy: 0.8600 - dense_9_accuracy: 0.8900
Loss on testing set= 1.7589656114578247


## Our Model works well on Training and Testing data

# The END